## Attempt to OCR and translate text to Spanish, would like to compare to tranlation if there is time

In [2]:
import torch
import numpy
import easyocr
from transformers import MarianMTModel, MarianTokenizer

/opt/miniconda3/envs/DL/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/miniconda3/envs/DL/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /opt/miniconda3/envs/DL/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/opt/miniconda3/envs/DL/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/miniconda3/envs/DL/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/miniconda3/envs/DL/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/miniconda3/envs/DL/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed

In [3]:
image_path = 'tri_image_eng.png'

In [4]:
import cv2

img = cv2.imread(image_path)

print(type(img))
print(img is None)

<class 'numpy.ndarray'>
False


In [5]:
import easyocr
import cv2

reader = easyocr.Reader(['en'], gpu=False)

img = cv2.imread(image_path)

results = reader.readtext(img)

print(results)

Using CPU. Note: This module is much faster with a GPU.


[([[np.int32(56), np.int32(35)], [np.int32(642), np.int32(35)], [np.int32(642), np.int32(110)], [np.int32(56), np.int32(110)]], 'A case of mistaken identity', np.float64(0.757877310475506)), ([[np.int32(54), np.int32(109)], [np.int32(466), np.int32(109)], [np.int32(466), np.int32(164)], [np.int32(54), np.int32(164)]], 'The first Triceratops fossis were a', np.float64(0.6524731045552304)), ([[np.int32(517), np.int32(121)], [np.int32(823), np.int32(121)], [np.int32(823), np.int32(165)], [np.int32(517), np.int32(165)]], 'of giant horns found near', np.float64(0.8108918619820832)), ([[np.int32(57), np.int32(157)], [np.int32(369), np.int32(157)], [np.int32(369), np.int32(201)], [np.int32(57), np.int32(201)]], 'Denver in 1887. However;', np.float64(0.7692000886406437)), ([[np.int32(428), np.int32(168)], [np.int32(490), np.int32(168)], [np.int32(490), np.int32(200)], [np.int32(428), np.int32(200)]], 'were', np.float64(0.9974524974822998)), ([[np.int32(486), np.int32(156)], [np.int32(774), np.

In [6]:
extracted_text = [res[1] for res in results]

print(extracted_text)

['A case of mistaken identity', 'The first Triceratops fossis were a', 'of giant horns found near', 'Denver in 1887. However;', 'were', 'mistakenly identified by', 'paleontologist 0.€. Marsh as belonging t0 a giant; extinct bison', '(a cousin to CUs mascot Ralphie) As more fossils were discovered;', 'Marsh realized his mistake; We now know those original giant horns', "belonged to Trceratops'", 'This cast (or replica) is based on bones from 10 different specimens', 'of Triceratops hordus; collected in Wyoming in the early 1890s. This', 'composite was exhibited as a single skeleton in 1905 in Washington', 'D.C. You are looking at a replica of the first mounted Triceratops in', 'the world.', 'pair', 'they']


In [7]:
formatted_text = "\n".join(extracted_text)

print(formatted_text)

A case of mistaken identity
The first Triceratops fossis were a
of giant horns found near
Denver in 1887. However;
were
mistakenly identified by
paleontologist 0.€. Marsh as belonging t0 a giant; extinct bison
(a cousin to CUs mascot Ralphie) As more fossils were discovered;
Marsh realized his mistake; We now know those original giant horns
belonged to Trceratops'
This cast (or replica) is based on bones from 10 different specimens
of Triceratops hordus; collected in Wyoming in the early 1890s. This
composite was exhibited as a single skeleton in 1905 in Washington
D.C. You are looking at a replica of the first mounted Triceratops in
the world.
pair
they


In [8]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-es"

tokenizer = MarianTokenizer.from_pretrained(model_name)

model = MarianMTModel.from_pretrained(
    model_name,
    use_safetensors=True
)

print("Model loaded successfully")

/opt/miniconda3/envs/DL/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Model loaded successfully


In [9]:
translated_lines = []

for line in formatted_text.split("\n"):
    # Skip empty lines
    if line.strip():
        inputs = tokenizer(line, return_tensors="pt", truncation=True)
        translated = model.generate(**inputs)
        translated_line = tokenizer.decode(translated[0], skip_special_tokens=True)
        translated_lines.append(translated_line)
    else:
        translated_lines.append("")

# Join back with line breaks
translated_text = "\n".join(translated_lines)

print(translated_text)

Un caso de identidad equivocada
Las primeras fossis de Triceratops fueron
de cuernos gigantes encontrados cerca
Denver en 1887. Sin embargo;
fueron
identificado erróneamente por
paleontólogo 0.€. Marsh como pertenencia a un gigante; bisonte extinto
(un primo de la mascota de CUs Ralphie) A medida que se descubrieron más fósiles;
Marsh se dio cuenta de su error; Ahora sabemos que los cuernos gigantes originales
perteneció a Trceratops'
Este molde (o réplica) se basa en huesos de 10 especímenes diferentes
de Triceratops hordus; recogido en Wyoming a principios de la década de 1890.
compuesto fue exhibido como un solo esqueleto en 1905 en Washington
D.C. Estás viendo una réplica de la primera Triceratops montada en
el mundo.
par
ellos


In [10]:
!pip install fpdf2


In [12]:
from fpdf import FPDF
from PIL import Image
import os

# --- 1. Open the image ---
im = Image.open(image_path)
img_width, img_height = im.size

# --- 2. Create PDF ---
pdf = FPDF()
pdf.add_page()
pdf.set_auto_page_break(auto=True, margin=15)

# --- 3. Add the image ---
pdf_width = pdf.w - 20  # 10mm margins each side
scale_factor = pdf_width / img_width
scaled_height = img_height * scale_factor
pdf.image(image_path, x=10, y=10, w=pdf_width, h=scaled_height)

# --- 4. Add a Unicode font ---
# Use any TTF font that supports Spanish/Unicode
# On macOS, this path works; adjust if using Windows/Linux
font_path = "/System/Library/Fonts/Supplemental/Arial Unicode.ttf"
if not os.path.exists(font_path):
    raise FileNotFoundError(f"Font not found: {font_path}. Use a TTF font that supports Unicode.")

pdf.add_font("ArialUnicode", "", font_path, uni=True)
pdf.set_font("ArialUnicode", size=12)

# --- 5. Add translated text below image ---
pdf.set_y(10 + scaled_height + 10)  # 10mm spacing below image
pdf.multi_cell(0, 8, translated_text)

# --- 6. Save PDF ---
pdf_output_path = "translation_report.pdf"
pdf.output(pdf_output_path)

print(f"PDF saved as {pdf_output_path}")

PDF saved as translation_report.pdf


/var/folders/nm/zhg7jf3n4gz76nqmv2nbzvrr0000gn/T/ipykernel_15438/643930345.py:27: DeprecationWarning: "uni" parameter is deprecated since v2.5.1 and will be removed in a future release
  pdf.add_font("ArialUnicode", "", font_path, uni=True)
